# Neural network 
1. Ustawienie losowych wag i biasów.
2. Przepuszenie otrzymanych wyników przez funkcje aktywacji aby dodać nielinowość.
3. Obliczenie błędu przewidywań wyników.
4. Uruchomienie propagacji wstecznej aby poprawić skuteczność przewidywanych wyników.
5. Trening modelu.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
def init_weight_bias():
    weights = []
    biases = []
    
    for _ in range(3):
        weights.append(np.random.rand(2, 2) - 0.5)
        biases.append(np.random.rand(1, 2) - 0.5)

    return weights, biases

Wyczytałem, że dobrze jest odjąć 0.5 od losowo wygenerowanej liczby bo stwarza to zakrec który zawiera liczby dodatnie jak i ujemne oraz wartości unikalne które sprzyjają lepszemu uczeniu się modelowi i nie powodują uczenia się w ten sam sposób na tych samych wagach i odchyleniach. Jest to optymalizacja problemu symmetry breaking. Odjęcie od bias wartości 0.5 pomaga w uniknięciu stronniczości w uczeniu maszynowym, jest to podobno standardowa inicjalizacja.

In [3]:
weights, biases = init_weight_bias()

print("Losowe wagi: ", weights)
print("Losowe biasy: ", biases)

Losowe wagi:  [array([[-0.03540673,  0.07771164],
       [ 0.27841192,  0.21259358]]), array([[ 0.35593504, -0.47568224],
       [-0.25641264,  0.33219778]]), array([[ 0.28354058, -0.40515167],
       [ 0.38376576,  0.01795235]])]
Losowe biasy:  [array([[-0.48383124, -0.09765069]]), array([[0.46089425, 0.48031995]]), array([[0.08582514, 0.12656794]])]


![alt text](images/warstwa.png "Layer")

In [4]:
# Abstrakcyjna wartwa bazowa
def layer(input_X, output_Y):
    input_X = None
    output_Y = None

    return input_X, output_Y

![alt text](images/sigmoid.png "funkcja aktywacji")

Zdecydowałem się na funkcje aktywacji sigmoid ponieważ wyczytałem, że jest idealna do klasyfkikacji binarnej 0,1 

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

![alt text](images/softmax.png "softmax")

In [6]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

Miałem nadzieję, że funkcja softmax dedykowana do używania na wyjściowej warstwie poprawi wyniki mojego modelu.

![alt text](images/propagacja_do_przodu.png "Propagacja do przodu")

![alt text](images/suma_wazona.png "Suma wazona")

In [7]:
def forward_propagation(x, weights, biases):
    # Pierwszy layer
    z1 = np.dot(x, weights[0]) + biases[0]
    result_activation_function = sigmoid(z1)
    # Drugi layer
    z2 = np.dot(result_activation_function, weights[1]) + biases[1]
    result_activation_function2 = sigmoid(z2)

    # Output layer
    z3 = np.dot(result_activation_function2, weights[2]) + biases[2]
    # output= sigmoid(z3)
    # Użyje jednak funkcji softmax na końcu sieci
    output = softmax(z3)


    return result_activation_function, result_activation_function2, output

Doszlismy do końca sieci więc należy policzyć straty aby poprawić wyniki przewidywań. 

In [8]:
def mse_loss(y_true, output_predict):
    y_true = y_true.reshape(-1, 1) # Reshape danych do tablic 2D
    output_predict = output_predict.reshape(-1, 1) # Reshape danych do tablic 2D
    output_error = mean_squared_error(y_true, output_predict)
    return output_error

Zdecydowałem się zastasować średni błęd kwadratowy ponieważ jest to dość popularna metoda i używałem tego wcześniej do sprawdzenia straty przy przewidywań modelu Linii Regresyjnej.

In [9]:
# Cost function: Binary cross-entropy loss
def compute_cost(y, y_hat):
    m = y.shape[0]                            # Number of samples
    y_hat = np.clip(y_hat, 1e-10, 1 - 1e-10)  # Avoid log(0) errors by clipping predictions

    # Vectorized implementation of binary cross-entropy loss
    cost = (-1 / m) * np.sum((y * np.log(y_hat)) + (1 - y) * np.log(1 - y_hat))
    return cost

![alt text](images/backpropagation.png "Backpropagation")

In [10]:
# def backward_propagation(x, output_error, weights, biases, learning_rate):

#     weights = np.array(weights)
#     biases = np.array(biases)

#     input_error = np.dot(output_error, weights.T)
#     weights_error = np.dot(x.T, output_error)

#     print(weights.shape)
#     print(weights_error.shape)
#     print(output_error.shape)

#     # Aktualizacja wag i baisow
#     for i in range(len(weights) - 1):
#         weights -= learning_rate * weights_error
#         biases -= learning_rate * output_error 

#     return weights, biases


Nie mogłem tego zrobić na swój własny sposób, bo mi się kształty nie zgadzały.

In [11]:
def backward_propagation(x, y, a1, a2, output, weights, biases, learning_rate):
    m = y.shape[0]
    error = output - y

    # Obliczanie gradientow propagacji wstecznej dla wartwy wyjscia
    dW3 = (1 / m) * np.dot(a2.T, error)
    db3 = (1 / m) * np.sum(error, axis=0, keepdims=True)

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz2 = np.dot(error, weights[2].T) * (a2 * (1 - a2))
    dW2 = (1 / m) * np.dot(a1.T, dz2)
    db2 = (1 / m) * np.sum(dz2, axis=0, keepdims=True)

    # Obliczanie gradientu propagacji wstecznej dla pierwszej wartwy
    dz1 = np.dot(dz2, weights[1].T) * (a1 * (1 - a1))
    dW1 = (1 / m) * np.dot(x.T, dz1)
    db1 = (1 / m) * np.sum(dz1, axis=0, keepdims=True)

    # Aktualizacja wag i biasów
    weights[2] -= learning_rate * dW3
    biases[2] -= learning_rate * db3
    weights[1] -= learning_rate * dW2
    biases[1] -= learning_rate * db2
    weights[0] -= learning_rate * dW1
    biases[0] -= learning_rate * db1

    return weights, biases

Nie potrafiłem sobie poradzić z implementacją propagacji wstecznej, dlatego użyłem kodu znalezionego na necie.

# Test sieci neuronowej

In [12]:
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
y_predict = np.array([[0], [1], [1], [0]])  

learning_rate = 0.01
epochs = 5000

for epoch in range(epochs):
    # Propagacja w przód
    layer1, layer2, output = forward_propagation(x_train, weights, biases)

    # Obliczenie kosztu straty po dojściu na koniec sieci
    # loss = mse_loss(y_predict, output)
    loss = compute_cost(y_predict, output)

    # Propagacja wsteczna
    update_weights, update_biases = backward_propagation(x_train, y_predict, layer1, layer2, output, weights, biases, learning_rate)

    if epoch % 1000 == 0:
        print(f"Iteracje/Epoki {epoch}, Strata : {loss:.4f}")
        print(f"Wejscia: {x_train.flatten()}, Przewidywania sieci: {output.flatten()}, rzeczywiste wyniki {y_predict.flatten()}")


Iteracje/Epoki 0, Strata : 2.2594
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.16187793 0.08814932 0.16177713 0.0880562  0.16194498 0.08822033
 0.16184599 0.08812812], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 1000, Strata : 2.2138
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12709967 0.11520068 0.13314945 0.12014929 0.12964056 0.11732973
 0.13540259 0.12202804], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 2000, Strata : 2.2096
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12061257 0.11995341 0.12676298 0.12584115 0.12485792 0.12402754
 0.12949052 0.1284539 ], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 3000, Strata : 2.2088
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.1210817  0.12107438 0.12604673 0.12592335 0.1249894  0.1248938
 0.12808074 0.1279099 ], rzeczywiste wyniki [0 1 1 0]
Iteracje/Epoki 4000, Strata : 2.2087
Wejscia: [0 0 0 1 1 0 1 1], Przewidywania sieci: [0.12170806 0.12173604 0.12582191 0.12578117 0.12514179 0.1251137
 0.12738207 0.12731527],

Podsumowanie: wyniki są beznadziejne, dalekie od dobrych przewidywań. Czy jest to spowodowane zanikiem spadku gradientu? Bo funkcja straty ma bardzo niewielkie spadki podczas większych iteracji. Jak by modelowi ciężko było zaktualizować lepsze wagi. Pamiętam, że czytałem taką informacje iż funkcja sigmoid ma tedencje do problemu zanikających gradientów, ale czy to na pewno to?